# Vast.ai Project Setup

**Quick setup for FSLR preprocessing on Vast.ai**

## Folder Structure
```
/workspace/
├── fslr-transformer-vs-iv3gru/     # Project code
├── data/
│   ├── raw/                        # Upload videos here
│   └── processed/                  # Output here
```


In [ ]:
import subprocess
import sys
from pathlib import Path

# Clone repository if not exists
project_root = Path("/workspace/fslr-transformer-vs-iv3gru")
if not project_root.exists():
    print("Cloning repository...")
    subprocess.run(["git", "clone", "https://github.com/jeipab/fslr-transformer-vs-iv3gru.git", str(project_root)], check=True)
else:
    print("Repository already exists, updating...")
    subprocess.run(["git", "-C", str(project_root), "pull"], check=True)

sys.path.insert(0, str(project_root))

# Install dependencies
print("Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(project_root / "requirements.txt")], check=True)

print("✅ Setup complete!")
print(f"📁 Project: {project_root}")
print(f"📁 Upload videos to: {project_root}/data/raw/")
print(f"📁 Output will go to: {project_root}/data/processed/")
print("\n🚀 Next: Run run_multi_preprocess.ipynb")
